In [196]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
def norm(word):
    
    p = morph.parse(word)[0]
    return p.normal_form, p.tag.POS

In [197]:
def sent_prep1(sent):

    #Normalization and stop-words delete from sent_list
    sent_list_norm = []
    label_list = []

    sent_list = []
    for x in sent:      
        #Split into words
        str_list = x.split()

        #List from sentenses (words in list)
        sent_list.append(str_list)


    for sent in sent_list:
        #print(sent)
        one_sent = ''

        if abs(float(sent[-1])) > 0.3:
            for x in sent[:-1]:
                #print(x)
                x_norm = norm(x)        
                if x_norm[1] not in ['PREP', 'PRCL', 'CONJ', 'NPRO']:
                    one_sent = one_sent + ' ' + x_norm[0]                
            weight = float(sent[-1])
            if weight > 0:
                weight_norm = 1
            else:
                weight_norm = -1
            one_sent = one_sent.lstrip()
            #print(one_sent)
            sent_list_norm.append(one_sent)
            label_list.append(weight_norm)
    #print(sent_list_norm)
    #print(label_list)
    return sent_list_norm, label_list

In [198]:
def sent_prep2(sent):

    #Normalization and stop-words delete from sent_list
    sent_list_norm = []
    sent_list = []
    for x in sent:
        
        #Split into words
        str_list = x.split()

        #List from sentenses (words in list)
        sent_list.append(str_list)
    for sent in sent_list:
        #print(sent)
        one_sent = ''        
        for x in sent[:-1]:
                #print(x)
                x_norm = norm(x)        
                if x_norm[1] not in ['PREP', 'PRCL', 'CONJ', 'NPRO']:
                    one_sent = one_sent + ' ' + x_norm[0]
        one_sent = one_sent.lstrip()
        #print(one_sent)
        sent_list_norm.append(one_sent)                
    return sent_list_norm

In [199]:
import re

#Make the dictionary from dic_ru file
with open('dic_ru.txt') as f:
    dic_list = f.readlines()
dic_list = [x.strip('\n') for x in dic_list]
dic = []
for x in dic_list:
    str_list = x.split()
    if len(str_list) == 3:
        #print(str_list[0], str_list[2])        
        #dic.append([str_list[0], float(str_list[2])])
        dic.append(str_list[0])
#print(dic)

In [200]:
#Ordered output to the file
with open ('dic_learn_ru.txt', 'w') as fp:    
    for x in dic:
        fp.write("%s %s\n" % (dic.index(x), x))

#Make the dictionary from sent_dic_ru file
with open('sent_dic_ru.txt') as f:
    sent = f.readlines()
sent = [x.strip('\n') for x in sent]                

#print(sent)    

In [201]:
sent_list_norm, label_list = sent_prep1(sent)

In [202]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

cv = CountVectorizer(vocabulary = dic)
vect_sent = cv.fit_transform(sent_list_norm)
vect_label = np.array(label_list)

In [203]:
n_train = len(sent_list_norm)
print('Number of sentences for training:', n_train)

Number of sentences for training: 75506


In [204]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf = SVC(kernel='linear')
clf.fit(vect_sent, vect_label)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [205]:
def raw_cut(text):

    #Delete digits    
    text = re.sub(r"\b\d+\b", "", text)

    #Split text into sentences and remove empty elements from list
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    sentences = list(filter(None, sentences))
    return sentences

In [210]:
text = input("Please enter some text: ")
sentences = raw_cut(text)
sent_list_norm = sent_prep2(sentences)
#cv = CountVectorizer(vocabulary = dic)
vect_sent = cv.fit_transform(sent_list_norm)
#print(vect_sent)
vect_label_test = []

for x in vect_sent:
    vect_label_test.extend(clf.predict(x).tolist())

for i in range(len(sentences)):
    print(sentences[i], vect_label_test[i])    

Please enter some text: Ужасно быстро вирус распространился в среде социально-незащищенных слоев населения. Солнце светило: день был яркий и безоблачный.
Ужасно быстро вирус распространился в среде социально-незащищенных слоев населения. -1
Солнце светило: день был яркий и безоблачный. 1
